In [4]:
%pip install --upgrade pip
%pip install --disable-pip-version-check \
    torch==1.13.1 \
    torchdata==0.5.1 --quiet

%pip install \
    transformers==4.27.2 \
    datasets==2.11.0 \
    evaluate==0.4.0 \
    rouge_score==0.1.2 \
    loralib==0.1.1 \
    peft==0.3.0 --quiet
!pip install -U accelerate
!pip install -U transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 17.6 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 23.1.2
    Uninstalling pip-23.1.2:
      Successfully uninstalled pip-23.1.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 887.5/887.5 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.6/4.6 MB 66.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 78.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 43.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 1.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 2.3.1+cu121 requires torch==2.3.1, but you have torch 1.13.1 which is incompatible.
torchtext 0.18.0 requires torch

In [1]:
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer,TrainingArguments, Trainer
import pandas as pd
from datasets import load_dataset

In [2]:
!kaggle datasets download -d jsonali2003/linguogen-text-generation-dataset

Dataset URL: https://www.kaggle.com/datasets/jsonali2003/linguogen-text-generation-dataset
License(s): Apache 2.0
  0% 0.00/10.0k [00:00<?, ?B/s]
100% 10.0k/10.0k [00:00<00:00, 14.3MB/s]


In [3]:
import zipfile
with zipfile.ZipFile('linguogen-text-generation-dataset.zip', 'r') as zip_ref:
    zip_ref.extractall('/content')

In [2]:
animal=pd.read_csv('/content/Animals.csv')
books=pd.read_csv('/content/Books.csv')
climate=pd.read_csv('/content/Climate.csv')
environment=pd.read_csv('/content/Environment.csv')
frnd=pd.read_csv('/content/Friends.csv')
hospital=pd.read_csv('/content/Hospital.csv')
movies=pd.read_csv('/content/Movies.csv')
religion=pd.read_csv('/content/Religion.csv')
School=pd.read_csv('/content/School.csv')
space=pd.read_csv('/content/Space.csv')


from datasets import Dataset, DatasetDict

animal=Dataset.from_pandas(animal)
books=Dataset.from_pandas(books)
climate=Dataset.from_pandas(climate)
environment=Dataset.from_pandas(environment)
frnd=Dataset.from_pandas(frnd)
hospital=Dataset.from_pandas(hospital)
movies=Dataset.from_pandas(movies)
religion=Dataset.from_pandas(religion)
School=Dataset.from_pandas(School)
space=Dataset.from_pandas(space)

dataset=DatasetDict({
'animal':animal,
'books':books,
'climate':climate,
'environment':environment,
'frnd':frnd,
'hospital':hospital,
'movies':movies,
'religion':religion,
'School':School
})

In [5]:
dataset

DatasetDict({
    animal: Dataset({
        features: ['Input Text', 'Target Text'],
        num_rows: 52
    })
    books: Dataset({
        features: ['Input Text', 'Target Text'],
        num_rows: 58
    })
    climate: Dataset({
        features: ['Input Text', 'Target Text'],
        num_rows: 51
    })
    environment: Dataset({
        features: ['Input Text', 'Target Text'],
        num_rows: 47
    })
    frnd: Dataset({
        features: ['Input Text', 'Target Text'],
        num_rows: 50
    })
    hospital: Dataset({
        features: ['Input Text', 'Target Text'],
        num_rows: 46
    })
    movies: Dataset({
        features: ['Input Text', 'Target Text'],
        num_rows: 50
    })
    religion: Dataset({
        features: ['Input Text', 'Target Text'],
        num_rows: 50
    })
    School: Dataset({
        features: ['Input Text', 'Target Text'],
        num_rows: 51
    })
})

In [3]:
tokenizer=GPT2Tokenizer.from_pretrained('gpt2')
model=GPT2LMHeadModel.from_pretrained('gpt2', pad_token_id=tokenizer.eos_token_id)



/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [16]:
import torch
import torch.nn as nn
tokenizer.pad_token = tokenizer.eos_token

def convert(example):
  start='Generate the next word for the given text '
  end='\n\nGenerated text '
  prompt=[start+x+end for x in example['Input Text']]
  example['input_ids'] = tokenizer(prompt, padding="max_length", truncation=True, return_tensors="pt").input_ids
  example['labels'] = tokenizer(example["Target Text"], padding="max_length", truncation=True, return_tensors="pt").input_ids
  return example

In [17]:

tdataset=dataset.map(convert,batched=True)
tdata=tdataset.remove_columns(['Input Text','Target Text'])


Map:   0%|          | 0/52 [00:00<?, ? examples/s]

Map:   0%|          | 0/58 [00:00<?, ? examples/s]

Map:   0%|          | 0/51 [00:00<?, ? examples/s]

Map:   0%|          | 0/47 [00:00<?, ? examples/s]

Map:   0%|          | 0/50 [00:00<?, ? examples/s]

Map:   0%|          | 0/46 [00:00<?, ? examples/s]

Map:   0%|          | 0/50 [00:00<?, ? examples/s]

Map:   0%|          | 0/50 [00:00<?, ? examples/s]

Map:   0%|          | 0/51 [00:00<?, ? examples/s]

In [18]:
from datasets import concatenate_datasets, load_dataset
all_splits = list(tdata.keys())
all_data = concatenate_datasets([tdata[split] for split in all_splits])

/usr/local/lib/python3.10/dist-packages/datasets/table.py:1381: FutureWarning: promote has been superseded by mode='default'.
  block_group = [InMemoryTable(cls._concat_blocks(list(block_group), axis=axis))]
/usr/local/lib/python3.10/dist-packages/datasets/table.py:1407: FutureWarning: promote has been superseded by mode='default'.
  table = cls._concat_blocks(blocks, axis=0)


In [20]:

all_data= all_data.filter(lambda example, index: index % 10 == 0, with_indices=True)
from sklearn.model_selection import train_test_split
df = all_data.to_pandas()
train_df, val_df = train_test_split(df, test_size=0.2, random_state=42)


train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)


Filter:   0%|          | 0/46 [00:00<?, ? examples/s]

In [11]:

df = all_data.to_pandas()


from sklearn.model_selection import train_test_split
train_df, val_df = train_test_split(df, test_size=0.2, random_state=42)


train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)

training_args = TrainingArguments(
    output_dir='/content/result',
    num_train_epochs=3,
    per_device_train_batch_size=1,           # Reduced batch size
    save_steps=1000,
    save_total_limit=3,
    prediction_loss_only=True,
    logging_dir='./logs',
    logging_steps=500,
    evaluation_strategy="epoch",
    warmup_steps=500,
    weight_decay=0.01,
    logging_first_step=True,
    learning_rate=2e-5,
    overwrite_output_dir=True,
)

# Use gradient accumulation to simulate a larger batch size
training_args.gradient_accumulation_steps = 2  # Accumulate gradients over 2 steps

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset
)

trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
We strongly recommend passing in an `attention_mask` since your input_ids may be padded. See https://huggingface.co/docs/transformers/troubleshooting#incorrect-output-when-padding-tokens-arent-masked.
You may ignore this warning if your `pad_token_id` (50256) is identical to the `bos_token_id` (50256), `eos_token_id` (50256), or the `sep_token_id` (None), and your input is not padded.


Epoch,Training Loss,Validation Loss
1,9.943800,10.063037
2,9.943800,3.186274
3,9.943800,0.422933


TrainOutput(global_step=54, training_loss=5.2725569407145185, metrics={'train_runtime': 2808.193, 'train_samples_per_second': 0.038, 'train_steps_per_second': 0.019, 'total_flos': 56439078912000.0, 'train_loss': 5.2725569407145185, 'epoch': 3.0})

In [12]:

model.save_pretrained('./fine-tuned-gpt2')
tokenizer.save_pretrained('./fine-tuned-gpt2')

('./fine-tuned-gpt2/tokenizer_config.json',
 './fine-tuned-gpt2/special_tokens_map.json',
 './fine-tuned-gpt2/vocab.json',
 './fine-tuned-gpt2/merges.txt',
 './fine-tuned-gpt2/added_tokens.json')

In [4]:
instruct=GPT2LMHeadModel.from_pretrained('./fine-tuned-gpt2')
tokenizer2=GPT2Tokenizer.from_pretrained('./fine-tuned-gpt2')

In [5]:

import evaluate
rouge=evaluate.load('rouge')

In [10]:
text=dataset['animal'][:30]["Input Text"]
original_text=dataset['animal'][:30]["Target Text"]
trained=[]
untrained=[]

for _,text in enumerate(text):

    input=tokenizer(text,return_tensors='pt')
    output1=tokenizer.decode(model.generate(input['input_ids'] , max_new_tokens=50)[0],skip_special_tokens=True)
    untrained.append(output1)
    output2=tokenizer2.decode(instruct.generate(input['input_ids'] , max_new_tokens=50)[0],skip_special_tokens=True)
    trained.append(output2)

The attention mask is not set and cannot be inferred from input because pad token is same as eos token.As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


In [7]:

zipped=list(zip(original_text , trained , untrained ))

In [11]:
df=pd.DataFrame(zipped,columns=['original','trained','untrained'])
df

,original,trained,untrained


In [9]:
df['trained']=df['trained'].apply(lambda x: x.split('\n')[0])
df['untrained']=df['untrained'].apply(lambda x: x.split('\n')[0])

In [ ]:
df

In [ ]:
from evaluate import load
rouge=load('rouge')
predictions = df['trained'].tolist()
references = df['original'].tolist()

rouge.add_batch(predictions=predictions, references=references)  # Add your data to the metric

result = rouge.compute()

predictions = df['untrained'].tolist()
references = df['original'].tolist()

rouge.add_batch(predictions=predictions, references=references)  # Add your data to the metric

result2 = rouge.compute()


In [ ]:
result

In [ ]:
result2

In [12]:
from peft import LoraConfig, get_peft_model, TaskType


In [13]:
lora_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,  # Causal Language Modeling for GPT-2
    inference_mode=False,
    r=8,  # Low-rank parameter
    lora_alpha=32,
    lora_dropout=0.2,
    bias="none"

)

In [14]:
modell=get_peft_model(model,lora_config)


/usr/local/lib/python3.10/dist-packages/peft/tuners/lora.py:240: UserWarning: fan_in_fan_out is set to False but the target module is `Conv1D`. Setting fan_in_fan_out to True.
  warnings.warn(


In [21]:

df = all_data.to_pandas()


from sklearn.model_selection import train_test_split
train_df, val_df = train_test_split(df, test_size=0.2, random_state=42)


train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)

training_args = TrainingArguments(
    output_dir='/content/resultt',
    num_train_epochs=3,
    per_device_train_batch_size=1,           # Reduced batch size
    save_steps=1000,
    save_total_limit=3,
    prediction_loss_only=True,
    logging_dir='./logs',
    logging_steps=500,
    evaluation_strategy="epoch",
    warmup_steps=500,
    weight_decay=0.01,
    logging_first_step=True,
    learning_rate=2e-5,
    overwrite_output_dir=True,
)

# Use gradient accumulation to simulate a larger batch size
training_args.gradient_accumulation_steps = 2  # Accumulate gradients over 2 steps

trainer = Trainer(
    model=modell,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset
)

trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
We strongly recommend passing in an `attention_mask` since your input_ids may be padded. See https://huggingface.co/docs/transformers/troubleshooting#incorrect-output-when-padding-tokens-arent-masked.
You may ignore this warning if your `pad_token_id` (50256) is identical to the `bos_token_id` (50256), `eos_token_id` (50256), or the `sep_token_id` (None), and your input is not padded.


Epoch,Training Loss,Validation Loss
1,10.081700,11.833814
2,10.081700,11.833342
3,10.081700,11.832496


TrainOutput(global_step=6, training_loss=10.03288745880127, metrics={'train_runtime': 239.1081, 'train_samples_per_second': 0.05, 'train_steps_per_second': 0.025, 'total_flos': 6292752039936.0, 'train_loss': 10.03288745880127, 'epoch': 3.0})

In [22]:
modell.save_pretrained('./fine-tuned-gpt2')
tokenizer.save_pretrained('./fine-tuned-gpt2')

('./fine-tuned-gpt2/tokenizer_config.json',
 './fine-tuned-gpt2/special_tokens_map.json',
 './fine-tuned-gpt2/vocab.json',
 './fine-tuned-gpt2/merges.txt',
 './fine-tuned-gpt2/added_tokens.json')

In [25]:
trainer.model.save_pretrained('./fine-tuned-gpt2-peft')
tokenizer.save_pretrained('./fine-tuned-gpt2-peft')

('./fine-tuned-gpt2/tokenizer_config.json',
 './fine-tuned-gpt2/special_tokens_map.json',
 './fine-tuned-gpt2/vocab.json',
 './fine-tuned-gpt2/merges.txt',
 './fine-tuned-gpt2/added_tokens.json')

In [26]:
from peft import PeftModel, PeftConfig

In [27]:
peft=PeftModel.from_pretrained(model,'./fine-tuned-gpt2',torch_dtype=torch.bfloat16,
                                       is_trainable=False)

In [29]:
text=dataset['animal'][:30]["Input Text"]
original_text=dataset['animal'][:30]["Target Text"]
trained=[]
untrained=[]

for _,text in enumerate(text):

    input=tokenizer(text,return_tensors='pt')
    output1=tokenizer.decode(model.generate(input['input_ids'] , max_new_tokens=50)[0],skip_special_tokens=True)
    untrained.append(output1)
    output2=tokenizer.decode(peft.generate(**input , max_new_tokens=50)[0],skip_special_tokens=True)
    trained.append(output2)

In [30]:
zipped=list(zip(original_text , trained , untrained ))
df=pd.DataFrame(zipped,columns=['original','trained','untrained'])
df['trained']=df['trained'].apply(lambda x: x.split('\n')[0])
df['untrained']=df['untrained'].apply(lambda x: x.split('\n')[0])

In [31]:
from evaluate import load
rouge=load('rouge')
predictions = df['trained'].tolist()
references = df['original'].tolist()

rouge.add_batch(predictions=predictions, references=references)  # Add your data to the metric

result = rouge.compute()

predictions = df['untrained'].tolist()
references = df['original'].tolist()

rouge.add_batch(predictions=predictions, references=references)  # Add your data to the metric

result2 = rouge.compute()


In [32]:
result

{'rouge1': 0.010095238095238096,
 'rouge2': 0.0,
 'rougeL': 0.01076190476190476,
 'rougeLsum': 0.009761904761904763}

In [33]:
result2

{'rouge1': 0.010095238095238096,
 'rouge2': 0.0,
 'rougeL': 0.01076190476190476,
 'rougeLsum': 0.009761904761904763}